## Load libraries

In [1]:
import numpy as np
import pandas as pd
import math
import pickle
import json
from gurobipy import *

## Functions

In [2]:
def pickle_load(file_dir):
  with open(file_dir, "rb") as fp:   # Unpickling
    return pickle.load(fp)

def json_load(file_dir):
    with open(file_dir) as f:
        return json.load(f)

In [3]:
data_dir = './data/'

# scenario number & distribution settings
scenario_num = 585
distribution_choice = 'normal'

# Sets directory
Bset_dir = data_dir + 'Bset.txt'
Bij_set_dir = data_dir + 'Bij_set.txt'
Sset_dir = data_dir+'Sset_' + str(scenario_num) + '.txt'

deterministic_params_dir = data_dir + 'deterministic_params.json'
capacity_i_dir = data_dir + 'capacity_i.json'
capacity_i_constant_dir = data_dir + 'capacity_i_constant.json'

# p_s_dir = data_dir + 'p_s_' + str(scenario_num) + '.json'
demScens_dir = data_dir + 'demScens_' + distribution_choice + '_' + str(scenario_num) + '.dict'

In [4]:
# Sets
Bset = pickle_load(Bset_dir)                                  # 22 Bike Stations
Bij_set = pickle_load(Bij_set_dir)                                  # Bike pairs                                        
Sset = pickle_load(Sset_dir)                                     # Scenario sets
deterministic_params = json_load(deterministic_params_dir)       # deterministic parameters

c = deterministic_params['c']                            # unit procurement cost
v_i = deterministic_params['v_i']                               # stock-out cost
w_i = deterministic_params['w_i']                              # time-waste cost 
t_ij = deterministic_params['t_ij']                    # unit transshipment cost 
capacity_i = json_load(capacity_i_dir)
# p_s = json_load(p_s_dir)
demScens = pickle_load(demScens_dir)


####################################################################
# Parameters
num_stage = 3
branch_factor = 8

# Size of the tree
num_leaf = branch_factor ** num_stage
num_nodes = int((1 - branch_factor ** (1 + num_stage)) / (1 - branch_factor)) # Note scenario_num is equal to num_nodes
num_non_leaf = num_nodes - num_leaf

# Calculate probability for each scenario
prob_leaf = [1/num_leaf] * num_leaf
prob_internal = []
for i in range(1, num_stage):
    prob_internal += [1/branch_factor ** i] * (branch_factor ** i)
    
# Separate leaf nodes and non_leaf nodes
Sset_non_leaf = Sset[:num_non_leaf]
Sset_internal = Sset[1:num_non_leaf]
Sset_leaf = Sset[num_non_leaf:]

# Create dictionary mapping from node to probability
p_leaf = {Sset_leaf[i]:prob_leaf[i] for i in range(num_leaf)}
p_internal = {Sset_internal[i]:prob_internal[i] for i in range(num_non_leaf-1)}

In [5]:
demScens

{('A', 'A', 'S0'): 0,
 ('A', 'A', 'S1'): 0,
 ('A', 'A', 'S2'): 0,
 ('A', 'A', 'S3'): 0,
 ('A', 'A', 'S4'): 0,
 ('A', 'A', 'S5'): 0,
 ('A', 'A', 'S6'): 0,
 ('A', 'A', 'S7'): 0,
 ('A', 'A', 'S8'): 0,
 ('A', 'A', 'S9'): 0,
 ('A', 'A', 'S10'): 0,
 ('A', 'A', 'S11'): 0,
 ('A', 'A', 'S12'): 0,
 ('A', 'A', 'S13'): 0,
 ('A', 'A', 'S14'): 0,
 ('A', 'A', 'S15'): 0,
 ('A', 'A', 'S16'): 0,
 ('A', 'A', 'S17'): 0,
 ('A', 'A', 'S18'): 0,
 ('A', 'A', 'S19'): 0,
 ('A', 'A', 'S20'): 0,
 ('A', 'A', 'S21'): 0,
 ('A', 'A', 'S22'): 0,
 ('A', 'A', 'S23'): 0,
 ('A', 'A', 'S24'): 0,
 ('A', 'A', 'S25'): 0,
 ('A', 'A', 'S26'): 0,
 ('A', 'A', 'S27'): 0,
 ('A', 'A', 'S28'): 0,
 ('A', 'A', 'S29'): 0,
 ('A', 'A', 'S30'): 0,
 ('A', 'A', 'S31'): 0,
 ('A', 'A', 'S32'): 0,
 ('A', 'A', 'S33'): 0,
 ('A', 'A', 'S34'): 0,
 ('A', 'A', 'S35'): 0,
 ('A', 'A', 'S36'): 0,
 ('A', 'A', 'S37'): 0,
 ('A', 'A', 'S38'): 0,
 ('A', 'A', 'S39'): 0,
 ('A', 'A', 'S40'): 0,
 ('A', 'A', 'S41'): 0,
 ('A', 'A', 'S42'): 0,
 ('A', 'A', 'S43'): 0

In [6]:
cpy = {}
for pair, val in demScens.items():
    cpy[((pair[0], pair[1]), pair[2])] = val
demScens = cpy

The formulation of the two-stage stochastic program problem i written as follows:

<b>First stage varibles:</b> <br>
$$x_i: \text{the number of bikes to assign to bike-station i} \in \text{B at the beginning of the service}$$
<b>Second stage variables: </b>
$$
\beta_{ij}: \text{Number of rented bikes from bike-station i to bike-station j in scenario s}\\
I_{is}^{+}: \text{Realized surplus of bikes at bike-station i in scenario s} \\
I_{ijs}^{-}: \text{Realized shortage of bikes at origin-destination pair i, j in scenario s}\\
\rho_{ijs}: \text{Number of redirected bikes from bike-station i to bike-station j in scenario s (in case of overflow)}\\
O_{is}^{+}: \text{Residual capacity at bike-station i in scenario s} \\
O_{is}^{-}: \text{Overflow at bike-station i in scenario s} \\
\tau_{ijs}: \text{Number of transshipped bikes from bike-station i to bike-station j in scenario s} \\
T_{is}^{+}: \text{Excess of bikes at bike-station i in scenario s} \\
T_is^{-}: \text{Lack of bikes at bike-station i in scenario s} \\
$$

<b>Formulation:</b>
$$
\begin{align*}
    \text{minimize: }   & c \sum_{i}^B x_i + \sum_{s=1}^S p_s\sum_{i=1}^B [v_i\sum_{j=1}^B I_{ijs}^{-} + w_iO_{is}^{-} + \sum_{j=1}^B t_{ij}\tau_{ijs}]\\
    \text{subject to: }   
        & x_{i} \le k_i, \forall i \in B &\text{(Bike Station Capacity)}\\
        & \beta_{ijs} = \xi_{ijs} - I_{ijs}^{-} , \forall i,j \in B, s \in S &\text{(rented bike successfully made from station i to j)}\\
        & I_{is}^{+} - \sum_{j=1}^B I_{ijs}^{-} = x_i - \sum_{j=1}^B \xi_{ijs} , \forall i \in B, s \in S &\text{(realized surplus and shortage)}\\
        & O_{is}^{+} - O_{is}^{-} = k_i - x_i + \sum_{j=1}^B \beta_{ijs} - \sum_{j=1}^B \beta_{ijs}, \forall i \in B, s \in S &\text{(residual capacity and overflow after rental)}\\
        & \sum_{j=1}^{B} \rho_{ijs} = O_{is}^{-}, \forall i \in B, s \in S &\text{(Redirection identifies station overflow)}\\
        & \sum_{j=1}^{B} \rho_{jis} \le O_{is}^{+}, \forall i \in B, s \in S &\text{(Sucessful redirection less than residual capacity)}\\
        & T_{is}^{+} - T_{is}^{-} = k_i - O_{is}^{+} + \sum_{j=1}^{B}\rho_{jis} - x_i, \forall i \in B, s \in S &\text{(Excess and Lack of bikes at stations)}\\
        & \sum_{j=1}^{B}\tau_{ijs} = T_{is}^{+}, \forall i \in B, s \in S &\text{(Transshipment of excessive bikes)}\\
        & \sum_{j=1}^{B}\tau_{jis} = T_{is}^{-}, \forall i \in B, s \in S &\text{(Transshipment fulfillment)}\\
        & x_i, I_{is}^{+}, O_{is}^{+}, O_{is}^{-}, T_{is}^{+}, T_{is}^{-} \in \mathbb{Z}^{+}, \forall i \in B, s \in S\\
        & \tau_{ijs}, \beta_{ijs}, \rho_{ijs}, I_{ijs}^{-} \in \mathbb{Z}^{+}, , \forall i,j \in B, s \in S\\
\end{align*}
$$

## Building Model

In [7]:
m = Model("2SP_ExtForm")

Using license file /Users/ruijian/gurobi.lic
Academic license - for non-commercial use only


### Set variables

In [8]:
# first stage variables
x = m.addVars(Bset, vtype=GRB.INTEGER, name='x')

# second stage variables
beta_ijs = m.addVars(Bset,Bset,Sset, vtype=GRB.INTEGER, name='beta_ijs')
I_is_surplus = m.addVars(Bset,Sset, vtype=GRB.INTEGER, name='I_ijs_surplus')
I_ijs_shortage = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='I_ijs_shortage')
rho_ijs = m.addVars(Bset,Bset, Sset, vtype=GRB.INTEGER, name='rho_ijs')
O_is_resCap = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_resCap')
O_is_overF = m.addVars(Bset, Sset, vtype=GRB.INTEGER, name='O_is_overF')
tau_ijs = m.addVars(Bset,Bset, Sset_leaf, vtype=GRB.INTEGER, name='tau_ijs')
T_is_excess = m.addVars(Bset, Sset_leaf, vtype=GRB.INTEGER, name='T_is_excess')
T_is_lack = m.addVars(Bset, Sset_leaf, vtype=GRB.INTEGER, name='T_is_lack')

m.setObjective(c*quicksum(x[i]for i in Bset) \
               + quicksum(p_internal[s]*quicksum(v_i * I_ijs_shortage.sum(i,'*',s)\
                                          + w_i * O_is_overF[(i,s)] for i in Bset) for s in Sset_internal) \
               + quicksum(p_leaf[s]*quicksum(v_i * I_ijs_shortage.sum(i,'*',s)\
                                          + w_i * O_is_overF[(i,s)] for i in Bset) for s in Sset_leaf)
               + quicksum(p_leaf[s]*quicksum(t_ij * tau_ijs.sum(i, '*', s) for i in Bset) for s in Sset_leaf))

### Set modelsense

In [9]:
m.modelSense = GRB.MINIMIZE

### Set constraints

In [10]:
m.addConstrs(
    (x[i] <= capacity_i[i] for i in Bset), name='assignment_capacity');

In [11]:
m.addConstrs(
    (beta_ijs[i, j, s] == demScens[(i, j), s] - I_ijs_shortage[i, j, s] for i in Bset for j in Bset for s in Sset_internal + Sset_leaf), name='actual_rental');

In [12]:
m.addConstrs(
    (I_is_surplus[i,s] - I_ijs_shortage.sum(i, '*', s) == x[i] - sum(demScens[(i,j),s] for j in Bset) for i in Bset for s in Sset_internal + Sset_leaf), \
        name='realized_surplus_shortage');

In [13]:
m.addConstrs((O_is_resCap[i,s] - O_is_overF[i,s] \
              == capacity_i[i] - x[i] + beta_ijs.sum(i, '*', s) - beta_ijs.sum('*', i, s) for i in Bset for s in Sset_internal + Sset_leaf),
            name='residual_overflow_after_rental');

In [14]:
m.addConstrs((rho_ijs.sum(i, '*', s) ==  O_is_overF[i, s] for i in Bset for s in Sset_internal + Sset_leaf),
            name='redirecting_overflow');

In [15]:
m.addConstrs((rho_ijs.sum('*', i, s) <=  O_is_resCap[i, s] for i in Bset for s in Sset_internal + Sset_leaf),
            name='successful_redictions');

In [16]:
m.addConstrs((T_is_excess[i, s] - T_is_lack[i, s] \
              == capacity_i[i] - O_is_resCap[i, s] + rho_ijs.sum('*', i, s) - x[i] for i in Bset for s in Sset_leaf),
            name='successful_redictions');

In [17]:
m.addConstrs((tau_ijs.sum(i, '*', s) ==  T_is_excess[i, s] for i in Bset for s in Sset_leaf),
            name='transshipment_excessive_bike');

In [18]:
m.addConstrs((tau_ijs.sum('*', i, s) ==  T_is_lack[i, s] for i in Bset for s in Sset_leaf),
            name='transshipment_fulfillment');

In [19]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 367862 rows, 1158388 columns and 2853862 nonzeros
Model fingerprint: 0x6fe25639
Variable types: 0 continuous, 1158388 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 319639 rows and 505693 columns (presolve time = 5s) ...
Presolve removed 319639 rows and 505693 columns
Presolve time: 9.43s
Presolved: 48223 rows, 652695 columns, 1725170 nonzeros
Variable types: 0 continuous, 652695 integer (135057 binary)
Found heuristic solution: objective 2779.3203125

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3453203e+03   0.000000e+00   1.490456e+04     13s
  114522    1.9449148e+03   0.000000e+00   4.288374e+04     15s
  352931    1

In [20]:
m.ObjVal

821.4296875

In [26]:
lst = []
for var in x:
    lst.append(x[var].X)

In [27]:
sum(lst)

219.0